In [ ]:
import org.apache.spark._
import org.apache.spark.SparkContext._
import org.apache.spark.rdd._

import scala.collection.JavaConversions._
import scala.util.control.Breaks._
import java.io._
import java.nio.file.{Files, Path, Paths}

import opennlp.tools.langdetect._
import opennlp.tools.lemmatizer.DictionaryLemmatizer
import opennlp.tools.postag.{POSModel, POSTaggerME}
import opennlp.tools.tokenize.{TokenizerME, TokenizerModel}
import opennlp.tools.util.Span


object Helpers {
    def using[A <: { def close(): Unit }, B](resource: A)(f: A => B): B =
        try {
            f(resource)
        } finally {
            resource.close()
        }
}

import Helpers._

val de : String = "de"
val en : String = "en"
val fr : String = "fr"
val langs : List[String] = List(de, en, fr)

//val conf = new SparkConf().setAppName("words")
//val sc = new SparkContext(conf)

def detectLang(line : String, dicts : scala.collection.mutable.Map[String, List[String]]) : Option[String] = {
    val langs = line.split(" ").flatMap(item => dicts.filter(_._2.exists(_.equalsIgnoreCase(item))).map(_._1))
                    .groupBy(f => f)
                    .map(g => (g._1, g._2.size))
    if(langs.isEmpty) None
    else Some(langs.maxBy(_._2)._1)
}

val stopwordsPerLang: scala.collection.mutable.Map[String, List[String]] = scala.collection.mutable.Map.empty 
// Stopwords per each lang, are all stored in RAM, because used for all supported languages detection & they're relatively small, quick to read.  

for(lang <- langs) { // main loop. TODO: refactor into function and call it async
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-stopwords.txt")) { source => 
    for (line <- source.getLines) {
      val list = stopwordsPerLang.getOrElse(lang, List.empty)
      stopwordsPerLang.update(lang, list:+line)  
    }
  }
}

val textfilesPathsPerLang: scala.collection.mutable.Map[String, List[String]] = scala.collection.mutable.Map.empty
val textfiles = Files.newDirectoryStream(Paths.get("notebooks/words/text-data"))
                     .filter(_.getFileName.toString.endsWith(".txt"))

for(filePath <- textfiles) {
  val file = filePath.toString
  using(scala.io.Source.fromFile(file)) { source => 
    val firstLine = source.getLines.next() // get single first line & detect language with it, TODO: better to use a few random lines in the middle of the text
    detectLang(firstLine, stopwordsPerLang) match  {
      case Some(lang) => {
        var list = textfilesPathsPerLang.getOrElse(lang, List.empty)
        textfilesPathsPerLang.update(lang, list:+file)        
      }
      case None => println("Language was not detected for file: $file")
    }                                              
  }
}
/*
case class Lemma(entry:String, default:String, pos:String)
object Lemma {
  def parse(line:String): Option[Lemma] = {
    line.split("\t") match { 
      case Array(f,s,t,_*) => Some(Lemma(f, s, t))
      case _ => None
    }
  }  
}
*/
// ?
def removeTextNoise(text:String) : String = {
  val removedNumbers = text.filter(!_.isDigit)
  // https://stackoverflow.com/questions/30074109/removing-punctuation-marks-form-text-in-scala-spark
  val removedWordsOfSizeLessEqual2AndPunctuation = removedNumbers.replaceAll("""([\p{Punct}&&[^.]]|\b\p{IsLetter}{1,2}\b)\s*""", "")
  // https://stackoverflow.com/questions/6198986/how-can-i-replace-non-printable-unicode-characters-in-java
  val removedUnicodes = removedWordsOfSizeLessEqual2AndPunctuation.replaceAll("""[\p{C}]""", "")
  val replacedEscapeSeqWithSpace =  removedUnicodes.replaceAll("""[\t\n\r\f\v]""", " ")
  replacedEscapeSeqWithSpace
}

/*
def readLemmas(lang:String) : scala.collection.mutable.ListBuffer[Lemma] = {
  var lemmas : scala.collection.mutable.ListBuffer[Lemma] = scala.collection.mutable.ListBuffer.empty  
  using(scala.io.Source.fromFile(s"notebooks/words/vocabs/$lang-lemmatizer.txt")) { source => 
    for (line <- source.getLines) {
      Lemma.parse(line) match { 
        case Some(lemma) => lemmas+=lemma
        case _ =>;
      }      
    }
  }
  lemmas
}
*/
/*
  def initTokenizer(lang:String) = {
    using(new FileInputStream(s"notebooks/words/vocabs/$lang-token.bin")) { stream =>
      val model : TokenizerModel = new TokenizerModel(stream)
      new TokenizerME(model);
    }
*/
class OpenNlpTokenizer(val model: TokenizerModel) {//extends Tokenizer {
  def this(lang:String) = this(OpenNlpTokenizer.loadDefaultModel(lang))

  val tokenizer = new TokenizerME(model)

  def tokenize(text: String): Seq[String] = {
    val positions = tokenizer.tokenizePos(text)
    val strings = positions.map {
      pos => text.substring(pos.getStart, pos.getEnd)
    }
    //assume(positions.length == strings.length)
    //for ((pos, string) <- (positions.iterator zip strings.iterator).toList)
    //yield new Token(string, pos.getStart)
    strings
  }  
}

object OpenNlpTokenizer {
  def loadDefaultModel(lang:String): TokenizerModel = {
    using(new FileInputStream(s"notebooks/words/vocabs/$lang-token.bin")) { stream =>
      new TokenizerModel(stream)
    }
  }
}
/*
object OpenNlpTokenizerMain extends TokenizerMain {
  val tokenizer = new OpenNlpTokenizer()
}
*/

//def removeStopWords(text:String) : String = {
//  ""
//}

for ((lang,textsPaths) <- textfilesPathsPerLang) { // main loop. TODO: refactor into function and call it async
   //val lemmas = readLemmas(lang)
  val tokenizer = new OpenNlpTokenizer(lang)
  
  for (paths <- textsPaths) {
    using(scala.io.Source.fromFile(paths)) { source => 
      val text = source.getLines.mkString
                                            
      val tokens = tokenizer.tokenize(text)
      
      val rdd: RDD[String] = sparkContext.parallelize(tokens)
      val tf: RDD[Vector] = new HashingTF().transform(rdd)
      tf.cache()
      val idf = new IDF().fit(tf)
      val tfidf: RDD[Vector] = idf.transform(tf)
      tfidf.foreach(x => println(x))                                            
    }
  }  
}

org.apache.spark.SparkException: Only one SparkContext may be running in this JVM (see SPARK-2243). To ignore this error, set spark.driver.allowMultipleContexts = true. The currently running SparkContext was created at:
org.apache.spark.sql.SparkSession$Builder.getOrCreate(SparkSession.scala:860)
$anon$1.reset(<console>:154)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
<init>(<console>:165)
<init>(<console>:200)
<init>(<console>:202)
.<init>(<console>:206)
.<clinit>(<console>)
.$print$lzycompute(<console>:7)
.$print(<console>:6)
$print(<console>)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java